In [1]:
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import random

In [2]:
# CUDA
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE.type

'cuda'

In [3]:
# Fix random seed
seed = 42
torch.manual_seed(seed)
if DEVICE.type == 'cuda':
	torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Pre-processing input data

In [4]:
def bits_to_MiB(row):
	# verify if has string ' MiB'
	if 'MiB' in str(row):
		row = row.replace(' MiB', '')
		row = float(row)
	else:
		row = float(row) / np.power(2, 20)
	return row


def MHz_to_GHz(row):
	# verify if has string ' GHz'
	if 'GHz' in str(row):
		row = row.replace(' GHz', '')
		# convert to float
		row = float(row)
	else:
		row = row.replace(' MHz', '')
		row = float(row) / 1000
	return row

In [5]:
results_df = pd.read_csv('../results/execution_time.csv')
results_savio_df = pd.read_csv('../results_savio/execution_time.csv')
results_df = pd.concat([results_df, results_savio_df], ignore_index=True)
# preprocessing
results_df['total_cpu_usage'] = results_df['total_cpu_usage'].str.replace('%', '').astype(float) / 100
results_df['max_ram_usage'] = results_df['max_ram_usage'] / 1024
results_df['l2_cache_size'] = results_df['l2_cache_size'].apply(bits_to_MiB)
results_df['l3_cache_size'] = results_df['l3_cache_size'].apply(bits_to_MiB)
results_df['ghz_actual_friendly'] = results_df['hz_actual_friendly'].apply(MHz_to_GHz)
results_df['ghz_advertised_friendly'] = results_df['hz_advertised_friendly'].str.replace('GHz', '').astype(float)
results_df = results_df.drop(columns=['hz_actual_friendly', 'hz_advertised_friendly', 'arch', 'vendor_id_raw'])

In [6]:
# Make the target dataset
target_df = results_df[['total_time', 'brand_raw', 'count', 'l2_cache_size', 'l3_cache_size', 'l2_cache_line_size', 'l2_cache_associativity', 'ghz_advertised_friendly', 'benchmark']].copy()
# Rename columns to *_target
target_df = target_df.rename(columns={
    'total_time': 'total_time_target',
    'brand_raw': 'brand_raw_target',
    'count': 'count_target',
    'l2_cache_size': 'l2_cache_size_target',
    'l3_cache_size': 'l3_cache_size_target',
    'l2_cache_line_size': 'l2_cache_line_size_target',
    'l2_cache_associativity': 'l2_cache_associativity_target',
    'ghz_advertised_friendly': 'ghz_advertised_friendly_target',
})

dataset_df = pd.merge(results_df, target_df, how='inner', on='benchmark')
dataset_df = dataset_df[dataset_df['brand_raw'] != dataset_df['brand_raw_target']]
dataset_df.head(2)

,total_time,total_cpu_usage,max_ram_usage,brand_raw,count,l2_cache_size,l3_cache_size,l2_cache_line_size,l2_cache_associativity,benchmark,ghz_actual_friendly,ghz_advertised_friendly,total_time_target,brand_raw_target,count_target,l2_cache_size_target,l3_cache_size_target,l2_cache_line_size_target,l2_cache_associativity_target,ghz_advertised_friendly_target
5,13.49,0.99,1435.851562,Intel(R) Core(TM) i5-10400 CPU @ 2.90GHz,12,1.5,12.0,256,6,KNP,4.0996,2.9,32.01,13th Gen Intel(R) Core(TM) i5-1335U,12,7.5,12.0,1280,7,2.496
6,13.49,0.99,1435.851562,Intel(R) Core(TM) i5-10400 CPU @ 2.90GHz,12,1.5,12.0,256,6,KNP,4.0996,2.9,19.09,13th Gen Intel(R) Core(TM) i5-1335U,12,7.5,12.0,1280,7,2.496


In [7]:
# remove one computer for testing
g_train = dataset_df[(dataset_df['brand_raw'] != '13th Gen Intel(R) Core(TM) i5-1335U') & (dataset_df['brand_raw_target'] != '13th Gen Intel(R) Core(TM) i5-1335U')].drop(columns=['benchmark','brand_raw', 'brand_raw_target'])
g_test = dataset_df[dataset_df['brand_raw_target'] == '13th Gen Intel(R) Core(TM) i5-1335U'].drop(columns=['benchmark','brand_raw', 'brand_raw_target'])

In [8]:
mm_df = dataset_df[dataset_df['benchmark']=='MATRIX_MULT'].drop(columns=['benchmark'])
# remove one computer for testing
mm_train = mm_df[(mm_df['brand_raw'] != '13th Gen Intel(R) Core(TM) i5-1335U') & (mm_df['brand_raw_target'] != '13th Gen Intel(R) Core(TM) i5-1335U')].drop(columns=['brand_raw', 'brand_raw_target'])
mm_test = mm_df[mm_df['brand_raw_target'] == '13th Gen Intel(R) Core(TM) i5-1335U'].drop(columns=['brand_raw', 'brand_raw_target'])

In [9]:
st_df = dataset_df[dataset_df['benchmark']!='MATRIX_MULT'].drop(columns=['benchmark','count'])
# remove one computer for testing
st_train = st_df[(st_df['brand_raw'] != '13th Gen Intel(R) Core(TM) i5-1335U') & (st_df['brand_raw_target'] != '13th Gen Intel(R) Core(TM) i5-1335U')].drop(columns=['brand_raw', 'brand_raw_target'])
st_test = st_df[st_df['brand_raw_target'] == '13th Gen Intel(R) Core(TM) i5-1335U'].drop(columns=['brand_raw', 'brand_raw_target'])

In [10]:
target = 'total_time_target'
features = mm_test.columns.copy().drop(target)
features_st = features.copy().drop(['count', 'count_target'])

In [11]:
# general data
## split data
X_g_train = g_train[features]
y_g_train = g_train[target]

X_g_test = g_test[features]
y_g_test = g_test[target]

## normalize data
scaler = StandardScaler()
X_g_train = scaler.fit_transform(X_g_train)
X_g_test = scaler.transform(X_g_test)

## convert to tensor
X_g_train = torch.tensor(X_g_train, dtype=torch.float32).unsqueeze(1)
X_g_test = torch.tensor(X_g_test, dtype=torch.float32).unsqueeze(1)
y_g_train = torch.tensor(y_g_train.values, dtype=torch.float32).view(-1, 1)
y_g_test = torch.tensor(y_g_test.values, dtype=torch.float32).view(-1, 1)

In [12]:
# single thread data
## split data
X_st_train = st_train[features_st]
y_st_train = st_train[target]

X_st_test = st_test[features_st]
y_st_test = st_test[target]

## normalize data
scaler = StandardScaler()
X_st_train = scaler.fit_transform(X_st_train)
X_st_test = scaler.transform(X_st_test)

## convert to tensor
X_st_train = torch.tensor(X_st_train, dtype=torch.float32).unsqueeze(1)
X_st_test = torch.tensor(X_st_test, dtype=torch.float32).unsqueeze(1)
y_st_train = torch.tensor(y_st_train.values, dtype=torch.float32).view(-1, 1)
y_st_test = torch.tensor(y_st_test.values, dtype=torch.float32).view(-1, 1)

In [13]:
# multi thread data
## split data
X_mm_train = mm_train[features]
y_mm_train = mm_train[target]

X_mm_test = mm_test[features]
y_mm_test = mm_test[target]

## normalize data
scaler = StandardScaler()
X_mm_train = scaler.fit_transform(X_mm_train)
X_mm_test = scaler.transform(X_mm_test)

## convert to tensor
X_mm_train = torch.tensor(X_mm_train, dtype=torch.float32).unsqueeze(1)
X_mm_test = torch.tensor(X_mm_test, dtype=torch.float32).unsqueeze(1)
y_mm_train = torch.tensor(y_mm_train.values, dtype=torch.float32).view(-1, 1)
y_mm_test = torch.tensor(y_mm_test.values, dtype=torch.float32).view(-1, 1)

In [14]:
if DEVICE.type == 'cuda':
	# move to DEVICE
	X_g_train = X_g_train.to(DEVICE)
	y_g_train = y_g_train.to(DEVICE)
	X_g_test = X_g_test.to(DEVICE)
	y_g_test = y_g_test.to(DEVICE)

	X_st_train = X_st_train.to(DEVICE)
	y_st_train = y_st_train.to(DEVICE)
	X_st_test = X_st_test.to(DEVICE)
	y_st_test = y_st_test.to(DEVICE)

	X_mm_train = X_mm_train.to(DEVICE)
	y_mm_train = y_mm_train.to(DEVICE)
	X_mm_test = X_mm_test.to(DEVICE)
	y_mm_test = y_mm_test.to(DEVICE)

# Model

In [15]:
class TransformerModel(nn.Module):
	def __init__(self, input_dim, model_dim, num_heads, num_layers, output_dim, dropout=0.1):
		super(TransformerModel, self).__init__()
		# layers
		self.embedding = nn.Linear(input_dim, model_dim)
		encoder_layer = nn.TransformerEncoderLayer(d_model=model_dim, nhead=num_heads, batch_first=True)
		self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
		self.fc = nn.Linear(model_dim, output_dim)
		self.dropout = nn.Dropout(dropout)
	
	def forward(self, x):
		x = self.embedding(x)
		x = self.dropout(x)
		x = self.transformer(x)
		x = self.fc(x.mean(dim=1))
		return x

In [16]:
def objective(trial: optuna.Trial, X_train, y_train, X_test, y_test, input_dim, output_dim):
	# Definimos los hiperparámetros a buscar
	num_heads = trial.suggest_int('num_heads', 1, 8)
	model_dim = trial.suggest_int('model_dim', num_heads * 4, num_heads * 64, step=num_heads)
	num_layers = trial.suggest_int('num_layers', 1, 6)
	dropout = trial.suggest_float('dropout', 0.1, 0.5)
	learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
	weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-2, log=True)
	num_epochs = trial.suggest_int('num_epochs', 10, 100)

	# model initialization 
	model = TransformerModel(input_dim, model_dim, num_heads, num_layers, output_dim, dropout)
	if DEVICE.type == 'cuda':
		model = model.to(DEVICE)
	criterion = nn.MSELoss()
	optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
	# training
	model.train()
	for epoch in range(num_epochs):
		optimizer.zero_grad()
		output = model(X_train)
		loss = criterion(output, y_train)
		loss.backward()
		optimizer.step()
	# evaluation
	model.eval()
	with torch.no_grad():
		predictions = model(X_test)
		val_loss = criterion(predictions, y_test)

		# trial.report(val_loss.item(), epoch+1)
		# if trial.should_prune():
		# 	raise optuna.TrialPruned()
	print(f"Trial: {trial.number} - Loss: {loss.item()} - Val Loss: {val_loss.item()}")
	return val_loss.item()

# Hyperparameters Optimization

In [17]:
n_trials = 25

## General

In [18]:
# configuration optuna
study_g = optuna.create_study(direction='minimize')
study_g.optimize(lambda trial: objective(trial, X_g_train, y_g_train, X_g_test, y_g_test, len(features), len(target)), n_trials=n_trials)

[I 2024-06-20 04:41:47,084] A new study created in memory with name: no-name-814156af-dfe2-4fc4-b4f4-f928c46c4894
/home/frnk65/resource-prediction-study/venv/lib/python3.10/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
/home/frnk65/resource-prediction-study/venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([7200, 1])) that is different to the input size (torch.Size([7200, 17])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/frnk65/resource-prediction-study/venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (to

Trial: 0 - Loss: 241.7632293701172 - Val Loss: 630.4725341796875


[I 2024-06-20 04:42:28,502] Trial 1 finished with value: 698.8194580078125 and parameters: {'num_heads': 4, 'model_dim': 120, 'num_layers': 6, 'dropout': 0.19228931201660393, 'learning_rate': 0.00022197998807286582, 'weight_decay': 0.0004353313038170108, 'num_epochs': 95}. Best is trial 0 with value: 630.4725341796875.


Trial: 1 - Loss: 281.3769836425781 - Val Loss: 698.8194580078125


/home/frnk65/resource-prediction-study/venv/lib/python3.10/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
[I 2024-06-20 04:42:30,786] Trial 2 finished with value: 800.7874145507812 and parameters: {'num_heads': 3, 'model_dim': 135, 'num_layers': 2, 'dropout': 0.256129713607558, 'learning_rate': 0.00012425991386710083, 'weight_decay': 1.643331370174554e-05, 'num_epochs': 32}. Best is trial 0 with value: 630.4725341796875.


...

Trial: 22 - Loss: 105.97310638427734 - Val Loss: 365.9410095214844


[I 2024-06-20 04:44:48,918] Trial 23 finished with value: 582.1925659179688 and parameters: {'num_heads': 3, 'model_dim': 75, 'num_layers': 5, 'dropout': 0.3318732618952987, 'learning_rate': 0.0011693304118209505, 'weight_decay': 0.00029637646728211984, 'num_epochs': 54}. Best is trial 20 with value: 228.31983947753906.


Trial: 23 - Loss: 215.77381896972656 - Val Loss: 582.1925659179688


[I 2024-06-20 04:45:00,433] Trial 24 finished with value: 687.0759887695312 and parameters: {'num_heads': 4, 'model_dim': 92, 'num_layers': 4, 'dropout': 0.27085389774497903, 'learning_rate': 0.00041080980023786126, 'weight_decay': 0.00011371790686247332, 'num_epochs': 72}. Best is trial 20 with value: 228.31983947753906.


Trial: 24 - Loss: 274.8428649902344 - Val Loss: 687.0759887695312


In [19]:
# Results
print(f'Número de pruebas: {len(study_g.trials)}')
trial = study_g.best_trial
print(f'Mejor prueba: {trial.number}')
print(f'Mejores parametros: {trial.params}')
print(f'Mejor valor de pérdida: {trial.value}')

Número de pruebas: 25
Mejor prueba: 20
Mejores parametros: {'num_heads': 3, 'model_dim': 54, 'num_layers': 4, 'dropout': 0.3228494095923632, 'learning_rate': 0.003047258572434952, 'weight_decay': 0.00020641330002257253, 'num_epochs': 71}
Mejor valor de pérdida: 228.31983947753906


## Single Thread

In [20]:
# configuration optuna
study_st = optuna.create_study(direction='minimize')
study_st.optimize(lambda trial: objective(trial, X_st_train, y_st_train, X_st_test, y_st_test, len(features_st), len(target)), n_trials=n_trials)

[I 2024-06-20 04:45:00,450] A new study created in memory with name: no-name-4c79815d-a5ee-415b-96d3-4285fe6ef8a8
/home/frnk65/resource-prediction-study/venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([5400, 1])) that is different to the input size (torch.Size([5400, 17])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/frnk65/resource-prediction-study/venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([675, 1])) that is different to the input size (torch.Size([675, 17])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
[I 2024-06-20 04:45:01,574] Trial 0 finished with value: 608.517578125 and parameters: {'num_heads': 1, 'model_dim':

Trial: 0 - Loss: 389.4541320800781 - Val Loss: 608.517578125


[I 2024-06-20 04:45:21,035] Trial 1 finished with value: 39.55455017089844 and parameters: {'num_heads': 7, 'model_dim': 140, 'num_layers': 4, 'dropout': 0.20452105085437045, 'learning_rate': 0.002707224949204147, 'weight_decay': 0.00018284144712738177, 'num_epochs': 96}. Best is trial 1 with value: 39.55455017089844.


Trial: 1 - Loss: 17.454801559448242 - Val Loss: 39.55455017089844


[I 2024-06-20 04:45:25,330] Trial 2 finished with value: 18.659408569335938 and parameters: {'num_heads': 5, 'model_dim': 245, 'num_layers': 4, 'dropout': 0.33568732964885606, 'learning_rate': 0.006292101996192198, 'weight_decay': 0.000120584928971844, 'num_epochs': 23}. Best is trial 2 with value: 18.659408569335938.


...

Trial: 22 - Loss: 17.552021026611328 - Val Loss: 33.20869827270508


[I 2024-06-20 04:50:26,542] Trial 23 finished with value: 34.671043395996094 and parameters: {'num_heads': 7, 'model_dim': 392, 'num_layers': 4, 'dropout': 0.3331321123864456, 'learning_rate': 0.003970774995179344, 'weight_decay': 0.0007031447975448214, 'num_epochs': 70}. Best is trial 2 with value: 18.659408569335938.


Trial: 23 - Loss: 17.545364379882812 - Val Loss: 34.671043395996094


[I 2024-06-20 04:50:43,253] Trial 24 finished with value: 36.572547912597656 and parameters: {'num_heads': 6, 'model_dim': 306, 'num_layers': 5, 'dropout': 0.2935946358334059, 'learning_rate': 0.0013868340757457137, 'weight_decay': 7.154081423987044e-05, 'num_epochs': 57}. Best is trial 2 with value: 18.659408569335938.


Trial: 24 - Loss: 17.559967041015625 - Val Loss: 36.572547912597656


In [21]:
# Results
print(f'Número de pruebas: {len(study_st.trials)}')
trial = study_st.best_trial
print(f'Mejor prueba: {trial.number}')
print(f'Mejores parametros: {trial.params}')
print(f'Mejor valor de pérdida en validación: {trial.value}')

Número de pruebas: 25
Mejor prueba: 2
Mejores parametros: {'num_heads': 5, 'model_dim': 245, 'num_layers': 4, 'dropout': 0.33568732964885606, 'learning_rate': 0.006292101996192198, 'weight_decay': 0.000120584928971844, 'num_epochs': 23}
Mejor valor de pérdida en validación: 18.659408569335938


## Multi Thread

In [22]:
# configuration optuna
study_mm = optuna.create_study(direction='minimize')
study_mm.optimize(lambda trial: objective(trial, X_mm_train, y_mm_train, X_mm_test, y_mm_test, len(features), len(target)), n_trials=n_trials)

[I 2024-06-20 04:50:43,268] A new study created in memory with name: no-name-61681b69-8d1c-449c-b50b-a0465cec8b60
/home/frnk65/resource-prediction-study/venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1800, 1])) that is different to the input size (torch.Size([1800, 17])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/frnk65/resource-prediction-study/venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([225, 1])) that is different to the input size (torch.Size([225, 17])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
[I 2024-06-20 04:50:45,874] Trial 0 finished with value: 638.0918579101562 and parameters: {'num_heads': 5, 'model_d

Trial: 0 - Loss: 113.08677673339844 - Val Loss: 638.0918579101562


[I 2024-06-20 04:50:46,739] Trial 1 finished with value: 928.3875122070312 and parameters: {'num_heads': 3, 'model_dim': 156, 'num_layers': 1, 'dropout': 0.42332718762588695, 'learning_rate': 0.0006234509490345264, 'weight_decay': 5.2549738725678564e-05, 'num_epochs': 83}. Best is trial 0 with value: 638.0918579101562.


Trial: 1 - Loss: 133.14291381835938 - Val Loss: 928.3875122070312


[I 2024-06-20 04:50:50,793] Trial 2 finished with value: 748.2271728515625 and parameters: {'num_heads': 7, 'model_dim': 329, 'num_layers': 4, 'dropout': 0.3930102912349217, 'learning_rate': 0.0008304519402535401, 'weight_decay': 4.497279652035178e-05, 'num_epochs': 46}. Best is trial 0 with value: 638.0918579101562.


...

Trial: 22 - Loss: 112.95305633544922 - Val Loss: 612.9329223632812


[I 2024-06-20 04:52:40,912] Trial 23 finished with value: 618.1771240234375 and parameters: {'num_heads': 5, 'model_dim': 275, 'num_layers': 5, 'dropout': 0.2706695732594715, 'learning_rate': 0.009971955672038523, 'weight_decay': 1.8403966093973846e-05, 'num_epochs': 70}. Best is trial 21 with value: 607.7455444335938.


Trial: 23 - Loss: 112.99073028564453 - Val Loss: 618.1771240234375


[I 2024-06-20 04:52:47,722] Trial 24 finished with value: 613.8899536132812 and parameters: {'num_heads': 4, 'model_dim': 240, 'num_layers': 6, 'dropout': 0.16909945254996686, 'learning_rate': 0.0052434734982383345, 'weight_decay': 0.00040634519986640236, 'num_epochs': 83}. Best is trial 21 with value: 607.7455444335938.


Trial: 24 - Loss: 112.92240142822266 - Val Loss: 613.8899536132812


In [23]:
# Results
print(f'Trials quantity: {len(study_mm.trials)}')
trial = study_mm.best_trial
print(f'Mejor prueba: {trial.number}')
print(f'Mejores parametros: {trial.params}')
print(f'Mejor valor de pérdida en validación: {trial.value}')

Trials quantity: 25
Mejor prueba: 21
Mejores parametros: {'num_heads': 6, 'model_dim': 336, 'num_layers': 5, 'dropout': 0.25241691549181033, 'learning_rate': 0.00987077757680359, 'weight_decay': 0.00011251132452506049, 'num_epochs': 74}
Mejor valor de pérdida en validación: 607.7455444335938


# Training

In [24]:
output_dim = 1

## General

In [25]:
input_dim = len(features)
# hyperparameters
num_heads = study_g.best_trial.params['num_heads']
model_dim = study_g.best_trial.params['model_dim']
num_layers = study_g.best_trial.params['num_layers']
dropout = study_g.best_trial.params['dropout']
lr = study_g.best_trial.params['learning_rate']
wd = study_g.best_trial.params['weight_decay']
num_epochs = study_g.best_trial.params['num_epochs']

study_g.best_trial.params

{'num_heads': 3,
 'model_dim': 54,
 'num_layers': 4,
 'dropout': 0.3228494095923632,
 'learning_rate': 0.003047258572434952,
 'weight_decay': 0.00020641330002257253,
 'num_epochs': 71}

In [26]:
# general model initialization
model_g = TransformerModel(input_dim, model_dim, num_heads, num_layers, output_dim, dropout)
if DEVICE.type == 'cuda':
	model_g = model_g.to(DEVICE)
criterion_g = nn.MSELoss()
optimizer_g = optim.AdamW(model_g.parameters(), lr=lr, weight_decay=wd)

model_g.train()

for epoch in range(num_epochs):
	optimizer_g.zero_grad()
	output = model_g(X_g_train)
	loss = criterion_g(output, y_g_train)
	loss.backward()
	optimizer_g.step()
	# validation
	if (epoch+1) % 10 == 0 or epoch == num_epochs-1:
		model_g.eval()
		with torch.no_grad():
			val_predictions = model_g(X_g_test)
			val_loss = criterion_g(val_predictions, y_g_test)
		print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, Val Loss: {val_loss.item()}')
		model_g.train()

Epoch 10/71, Loss: 239.3172149658203, Val Loss: 613.6309814453125
Epoch 20/71, Loss: 190.65248107910156, Val Loss: 524.8025512695312
Epoch 30/71, Loss: 142.29408264160156, Val Loss: 434.852294921875
Epoch 40/71, Loss: 101.49819946289062, Val Loss: 348.8166198730469
Epoch 50/71, Loss: 72.3261489868164, Val Loss: 274.54254150390625
Epoch 60/71, Loss: 55.270992279052734, Val Loss: 216.94171142578125
Epoch 70/71, Loss: 48.282691955566406, Val Loss: 178.2835235595703
Epoch 71/71, Loss: 47.95528030395508, Val Loss: 175.4420928955078


## Single Thread

In [27]:
input_dim = len(features_st)
# hyperparameters
num_heads = study_st.best_trial.params['num_heads']
model_dim = study_st.best_trial.params['model_dim']
num_layers = study_st.best_trial.params['num_layers']
dropout = study_st.best_trial.params['dropout']
lr = study_st.best_trial.params['learning_rate']
wd = study_st.best_trial.params['weight_decay']
num_epochs = study_st.best_trial.params['num_epochs']
study_st.best_trial.params

{'num_heads': 5,
 'model_dim': 245,
 'num_layers': 4,
 'dropout': 0.33568732964885606,
 'learning_rate': 0.006292101996192198,
 'weight_decay': 0.000120584928971844,
 'num_epochs': 23}

In [28]:
# single thread model initialization
model_st = TransformerModel(input_dim, model_dim, num_heads, num_layers, output_dim, dropout)
if DEVICE.type == 'cuda':
	model_st = model_st.to(DEVICE)
criterion_st = nn.MSELoss()
optimizer_st = optim.AdamW(model_st.parameters(), lr=lr, weight_decay=wd)

model_st.train()

for epoch in range(num_epochs):
	optimizer_st.zero_grad()
	output = model_st(X_st_train)
	loss = criterion_st(output, y_st_train)
	loss.backward()
	optimizer_st.step()
	# validation
	if (epoch+1) % 10 == 0 or epoch == num_epochs-1:
		model_st.eval()
		with torch.no_grad():
			val_predictions = model_st(X_st_test)
			val_loss = criterion_st(val_predictions, y_st_test)
		print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, Val Loss: {val_loss.item()}')
		model_st.train()

Epoch 10/23, Loss: 19.4830379486084, Val Loss: 36.49971008300781
Epoch 20/23, Loss: 22.07948875427246, Val Loss: 28.931926727294922
Epoch 23/23, Loss: 20.814666748046875, Val Loss: 56.268009185791016


## Multi Thread

In [29]:
input_dim = len(features)
# hyperparameters
num_heads = study_mm.best_trial.params['num_heads']
model_dim = study_mm.best_trial.params['model_dim']
num_layers = study_mm.best_trial.params['num_layers']
dropout = study_mm.best_trial.params['dropout']
lr = study_mm.best_trial.params['learning_rate']
wd = study_mm.best_trial.params['weight_decay']
num_epochs = study_mm.best_trial.params['num_epochs']

study_mm.best_trial.params

{'num_heads': 6,
 'model_dim': 336,
 'num_layers': 5,
 'dropout': 0.25241691549181033,
 'learning_rate': 0.00987077757680359,
 'weight_decay': 0.00011251132452506049,
 'num_epochs': 74}

In [30]:
# multi thread model initialization
model_mm = TransformerModel(input_dim, model_dim, num_heads, num_layers, output_dim, dropout)
if DEVICE.type == 'cuda':
	model_mm = model_mm.to(DEVICE)
criterion_mm = nn.MSELoss()
optimizer_mm = optim.AdamW(model_mm.parameters(), lr=lr, weight_decay=wd)

model_mm.train()

for epoch in range(num_epochs):
	optimizer_mm.zero_grad()
	output = model_mm(X_mm_train)
	loss = criterion_mm(output, y_mm_train)
	loss.backward()
	optimizer_mm.step()
	# validation
	if (epoch+1) % 10 == 0 or epoch == num_epochs-1:
		model_mm.eval()
		with torch.no_grad():
			val_predictions = model_mm(X_mm_test)
			val_loss = criterion_mm(val_predictions, y_mm_test)
		print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, Val Loss: {val_loss.item()}')
		model_mm.train()

Epoch 10/74, Loss: 113.2838363647461, Val Loss: 637.9877319335938
Epoch 20/74, Loss: 113.93785095214844, Val Loss: 561.0780029296875
Epoch 30/74, Loss: 112.93811798095703, Val Loss: 608.9549560546875
Epoch 40/74, Loss: 113.05597686767578, Val Loss: 631.4415283203125
Epoch 50/74, Loss: 112.85626983642578, Val Loss: 618.8742065429688
Epoch 60/74, Loss: 113.09892272949219, Val Loss: 609.8304443359375
Epoch 70/74, Loss: 112.87665557861328, Val Loss: 609.0673828125
Epoch 74/74, Loss: 113.03215789794922, Val Loss: 615.2300415039062


# Conclusion
Queda trabajo que hacer en la red, además de conseguir más datos para un entrenamiento más robusto. Queda por ahora descartado el uso de solo un modelo para multi-threading y single-threading, ya que el modelo tiene más del triple de *loss*.